# 🍀オリジナルLoRA の作成_🌱お試しサンプル版（SD1.5版）
### ※Lora Trainer by Zasuko Michikusa
🌱This is based on the work of [Kohya-ss](https://github.com/kohya-ss/sd-scripts) and [Linaqruf](https://colab.research.google.com/github/Linaqruf/kohya-trainer/blob/main/kohya-LoRA-dreambooth.ipynb).and [Hollowstrawberry](https://github.com/hollowstrawberry/kohya-colab) Thank you!


In [ ]:
#@title # 🌎＜Step1＞ Google Driveと接続

#@markdown # 🥚はじめにやること➊❷
#@markdown #＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝
#@markdown #➊🐣マイドライブに階層フォルダを構成
#@markdown ### 　・予め下記🍀マークのフォルダと必要ファイルを用意しましょう！
#@markdown ###～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～
#@markdown ###＜🌎GoogleDrive＞
#@markdown ###📁マイドライブ
#@markdown ###└📁checkpoint　　　← 🍀ベースに使用するCheckpointモデルを格納
#@markdown ###└📁train_img　　　 ← 🍀学習用の画像とキャプションを格納
#@markdown ###└📁Loras 　　　　　←(※1)　作成するLoRAプロジェクト単位でフォルダ管理します
#@markdown ###＿└📁project_name　←(※2)　❷で入力した名前になる（完成したLoRAファイルの名前にもなる）
#@markdown ###＿＿└📁output　　　← (※3)　完成した LoRAファイル が格納される
#@markdown ###＿＿└📁dataset　  　 ← (※4)　学習に使う画像とキャプションを入れるフォルダ
from google.colab import drive
drive.mount('/content/drive')

import os
import re
import toml
from time import time
from IPython.display import Markdown, display

# これらは過去の実行からの情報を保持します
if "model_url" in globals():
    old_model_url = model_url
else:
    old_model_url = None
if "dependencies_installed" not in globals():
    dependencies_installed = False
if "model_file" not in globals():
    model_file = None

# 他のセルによって設定される可能性のある変数。これらはレガシーな変数です。
if "custom_dataset" not in globals():
    custom_dataset = None
if "override_dataset_config_file" not in globals():
    override_dataset_config_file = None
if "override_config_file" not in globals():
    override_config_file = None
if "optimizer" not in globals():
    optimizer = "AdamW8bit"
if "optimizer_args" not in globals():
    optimizer_args = None
if "continue_from_lora" not in globals():
    continue_from_lora = ""
if "weighted_captions" not in globals():
    weighted_captions = False
if "adjust_tags" not in globals():
    adjust_tags = False
if "keep_tokens_weight" not in globals():
    keep_tokens_weight = 1.0

COLAB = True  # 低RAMモードのための設定
XFORMERS = True
SOURCE = "https://github.com/uYouUs/sd-scripts"
COMMIT = None
BETTER_EPOCH_NAMES = True
LOAD_TRUNCATED_IMAGES = True

#@markdown ###～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～～
#@markdown #【詳しい説明】
#@markdown ####💡「📁Loras」以下は🌎＜Step1＞の処理後に自動生成されます
#@markdown ##### ※1 名前を付けたプロジェクト単位で自動的にフォルダ管理します
#@markdown ##### ※2「project_name」フォルダ名を付けて下記フォルダを格納
#@markdown ##### ※3「output」フォルダだけ作成しておく
#@markdown ##### ※4「dateset」に学習用画像とキャプションファイルを格納
#@markdown ##### ・学習用画像の画像枚数は「20枚～50枚」（解像度：512×512）
#@markdown ##### ・キャプションファイルは、予め「dataset-tag-editor」で作成してください。

#@markdown #ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー

#@markdown #❷🐔作成する「LoRA名」の設定
#@markdown ### 　・自作する「LoLA名」を入力してください。（「.」スペース等は使用不可）
project_name = "ここに任意のLoRAの名前を入力"  # @param {type:"string"}
project_name = project_name.strip()

folder_structure = "カテゴリごとに整理 (MyDrive/Loras/project_name/dataset)"  # @param ["カテゴリごとに整理 (MyDrive/Loras/project_name/dataset)"]
#@markdown #＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝

#@markdown ※Stable Diffusion 2系モデルを使用する場合はチェックを入れてください。
custom_model_is_based_on_sd2 = False  # @param {type:"boolean"}


def create_folders():
    # Google Driveがマウントされていない場合はマウント
    if COLAB and not os.path.exists('/content/drive'):
        from google.colab import drive
        print("📂 Google Driveに接続中...")
        drive.mount('/content/drive')

    # 作成したいフォルダのパスをリストで指定
    folders_to_create = [
        f'/content/drive/MyDrive/Loras/{project_name}',  # プロジェクトフォルダ
        f'/content/drive/MyDrive/Loras/{project_name}/dataset',  # データセットフォルダ
        f'/content/drive/MyDrive/Loras/{project_name}/output',  # 出力フォルダ
        f'/content/drive/MyDrive/Loras/{project_name}/config'  # コンフィグフォルダ
    ]

    # フォルダを作成する
    for folder in folders_to_create:
        os.makedirs(folder, exist_ok=True)
        print(f"📂 フォルダ {folder} が作成されました")

# 関数を実行
create_folders()

print("🌎GoogleDriveとの接続が完了しました！ 🌱Step➋へ進んでください")

In [ ]:

#@title # 🌱＜Step2＞ 学習をはじめる前の設定

#@markdown # 📔学習に関する設定➊❷③
#@markdown #＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝
#@markdown #➊「📚学習用の画像」＆「📑キャプション」の格納
#@markdown ### 　・「📂dataset」内に「学習用の画像」と「キャプション」をアップロードしてください。
#@markdown ### 　※◀のフォルダリストリストを開いて、対象の「📂dataset」を右クリック

#@markdown #ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー

#@markdown #❷ 🔀学習に使用するモデルの指定
#@markdown ### 　・GoogleDrive内にある、ベースモデル（Checkpoint）のパスを指定してください。
checkpoint_model_path = "/content/drive/MyDrive/Original_Lora/checkpoint/～～～～～～～～～～～～"  # @param {type:"string"}

# モデルパスの判定
model_url = checkpoint_model_path  # Google Drive上のパスを指定

# モデルファイルが存在するかチェック
if not os.path.exists(model_url):
    raise FileNotFoundError(f"モデルファイルが見つかりません: {model_url}")

# モデルをロードする処理
model_file = model_url  # そのままモデルファイルのパスを利用

# パスやファイルが正しいかどうかを確認するための簡単な出力
print(f"使用するモデルのパス: {model_url}")

#@markdown #ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー

#@markdown #③ ⚙️各種パラメータ設定（設定省略）
#@markdown ### 　♻️ 🌱お試し版なので各種、私のオススメ固定値で設定済みです。
# 固定された学習設定
resolution = 512  # 解像度を512に固定
flip_aug = False  # タグのシャッフルをしない
shuffle_tags = True  # タグシャッフルはTrueで固定

shuffle_caption = shuffle_tags
activation_tags = 1  # アクティベーションタグは1に固定
keep_tokens = int(activation_tags)

# 学習ステップに関する設定
num_repeats = 10  # 繰り返し回数は10に固定
max_train_epochs = 10  # エポック数を10に固定
max_train_steps = None  # ステップ数は使用しない

save_every_n_epochs = 10  # 10エポックごとに保存
keep_only_last_n_epochs = 10  # 最新の10エポックのみ保持

train_batch_size = 2  # バッチサイズを2に固定

# 学習率とスケジューラの設定
unet_lr = 5e-4  # UNetの学習率を5e-4に固定
text_encoder_lr = 1e-4  # テキストエンコーダの学習率を1e-4に固定

lr_scheduler = "cosine_with_restarts"  # スケジューラは`cosine_with_restarts`に固定
lr_scheduler_num_cycles = 3  # サイクル数を3に固定
lr_scheduler_power = 0  # ポリノミアルのパワーを0に固定

lr_warmup_ratio = 0.05  # ウォームアップ比率を5%に固定
lr_warmup_steps = 0  # ウォームアップステップは使用しない

min_snr_gamma = True  # SNRガンマを有効にする
min_snr_gamma_value = 5.0  # SNRガンマの値を5.0に固定

# LoRAのタイプとネットワーク設定
lora_type = "LoRA"  # LoRAタイプを"LoRA"に固定
network_dim = 16  # ネットワークの次元を16に固定
network_alpha = 8  # ネットワークのアルファを8に固定

# LoConの追加設定
conv_dim = 8  # LoConの畳み込み次元を8に固定
conv_alpha = 4  # LoConのアルファを4に固定

network_module = "networks.lora"
network_args = None
if lora_type.lower() == "locon":
    network_args = [f"conv_dim={conv_dim}", f"conv_alpha={conv_alpha}"]

#@markdown #＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝
#@markdown # 👌 ↑ここまで設定できたら準備完了↑
#@markdown 🚩🌱＜Step2＞の（▶️）実行ボタンを押すと、LoRAの学習が開始されます。 <p>



# ここから下が学習コマンド

if optimizer.lower() == "prodigy" or "dadapt" in optimizer.lower():
  if override_values_for_dadapt_and_prodigy:
    unet_lr = 0.5
    text_encoder_lr = 0.5
    lr_scheduler = "constant_with_warmup"
    lr_warmup_ratio = 0.05
    network_alpha = network_dim

  if not optimizer_args:
    optimizer_args = ["decouple=True","weight_decay=0.01","betas=[0.9,0.999]"]
    if optimizer == "Prodigy":
      optimizer_args.extend(["d_coef=2","use_bias_correction=True"])
      if lr_warmup_ratio > 0:
        optimizer_args.append("safeguard_warmup=True")
      else:
        optimizer_args.append("safeguard_warmup=False")

root_dir = "/content" if COLAB else "~/Loras"
deps_dir = os.path.join(root_dir, "deps")
repo_dir = os.path.join(root_dir, "kohya-trainer")

if "/Loras" in folder_structure:
  main_dir      = os.path.join(root_dir, "drive/MyDrive/Loras") if COLAB else root_dir
  log_folder    = os.path.join(main_dir, "_logs")
  config_folder = os.path.join(main_dir, project_name)
  images_folder = os.path.join(main_dir, project_name, "dataset")
  output_folder = os.path.join(main_dir, project_name, "output")
else:
  main_dir      = os.path.join(root_dir, "drive/MyDrive/lora_training") if COLAB else root_dir
  images_folder = os.path.join(main_dir, "datasets", project_name)
  output_folder = os.path.join(main_dir, "output", project_name)
  config_folder = os.path.join(main_dir, "config", project_name)
  log_folder    = os.path.join(main_dir, "log")

config_file = os.path.join(config_folder, "training_config.toml")
dataset_config_file = os.path.join(config_folder, "dataset_config.toml")
accelerate_config_file = os.path.join(repo_dir, "accelerate_config/config.yaml")

def install_dependencies():
  os.chdir(root_dir)
  !git clone {SOURCE} {repo_dir}
  os.chdir(repo_dir)
  if COMMIT:
    !git reset --hard {COMMIT}
  !wget https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/train_network_wrapper.py -q -O train_network_wrapper.py
  !wget https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/dracula.py -q -O dracula.py

  !apt -y update -qq
  !apt -y install aria2 -qq
  !pip install accelerate==0.25.0 transformers==4.36.2 diffusers[torch]==0.25.0 ftfy==6.1.1 \
    opencv-python==4.8.1.78 einops==0.7.0 pytorch-lightning==1.9.0 bitsandbytes==0.43.0 \
    prodigyopt==1.0 lion-pytorch==0.0.6 tensorboard safetensors==0.4.2 altair==4.2.2 \
    easygui==0.98.3 toml==0.10.2 voluptuous==0.13.1 huggingface-hub==0.20.1 imagesize==1.4.1 \
    rich==13.7.1 torch==2.4.1+cu121 triton
  !pip install -e .
  if XFORMERS:
    !pip install xformers==0.0.28.post1

  # patch kohya for minor stuff
  if COLAB:
    !sed -i "s@cpu@cuda@" library/model_util.py # low ram
  if LOAD_TRUNCATED_IMAGES:
    !sed -i 's/from PIL import Image/from PIL import Image, ImageFile\nImageFile.LOAD_TRUNCATED_IMAGES=True/g' library/train_util.py # fix truncated jpegs error
  if BETTER_EPOCH_NAMES:
    !sed -i 's/{:06d}/{:02d}/g' library/train_util.py # make epoch names shorter
    !sed -i 's/"." + args.save_model_as)/"-{:02d}.".format(num_train_epochs) + args.save_model_as)/g' train_network.py # name of the last epoch will match the rest

  from accelerate.utils import write_basic_config
  if not os.path.exists(accelerate_config_file):
    write_basic_config(save_location=accelerate_config_file)

  os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
  os.environ["BITSANDBYTES_NOWELCOME"] = "1"
  os.environ["SAFETENSORS_FAST_GPU"] = "1"

def validate_dataset():
    global lr_warmup_steps, lr_warmup_ratio, caption_extension, keep_tokens, keep_tokens_weight, weighted_captions, adjust_tags
    supported_types = (".png", ".jpg", ".jpeg", ".webp", ".bmp")

    print("\n💿 データセットを確認中...")

    # プロジェクト名のチェック
    if not project_name.strip() or any(c in project_name for c in " .()\"'\\/"):
        print("👀 エラー: 有効なプロジェクト名を指定してください。")
        return

    # データセットフォルダの存在確認
    if not os.path.exists(images_folder):
        print(f"👀 エラー: フォルダ {images_folder} が存在しません。")
        return
    else:
        print(f"データセットフォルダ {images_folder} は存在します。")

    if custom_dataset:
        try:
            datconf = toml.loads(custom_dataset)
            datasets = [d for d in datconf["datasets"][0]["subsets"]]
        except:
            print(f"👀 エラー: カスタムデータセットが無効かエラーを含んでいます！テンプレートを確認してください。")
            return
        reg = [d.get("image_dir") for d in datasets if d.get("is_reg", False)]
        datasets_dict = {d["image_dir"]: d["num_repeats"] for d in datasets}
        folders = datasets_dict.keys()
        files = [f for folder in folders for f in os.listdir(folder)]
        images_repeats = {folder: (len([f for f in os.listdir(folder) if f.lower().endswith(supported_types)]), datasets_dict[folder]) for folder in folders}
    else:
        reg = []
        folders = [images_folder]
        files = os.listdir(images_folder)
        images_repeats = {images_folder: (len([f for f in files if f.lower().endswith(supported_types)]), num_repeats)}

    for folder in folders:
        if not os.path.exists(folder):
            print(f"👀 エラー: フォルダ {folder.replace('/content/drive/', '')} が存在しません。")
            return
    for folder, (img, rep) in images_repeats.items():
        if not img:
            print(f"👀 エラー: フォルダ {folder.replace('/content/drive/', '')} が空です。")
            return
    for f in files:
        if not f.lower().endswith((".txt", ".npz")) and not f.lower().endswith(supported_types):
            print(f"👀 エラー: データセット内に無効なファイルがあります: \"{f}\"。中止します。")
            return

    if not [txt for txt in files if txt.lower().endswith(".txt")]:
        caption_extension = ""
    if continue_from_lora and not (continue_from_lora.endswith(".safetensors") and os.path.exists(continue_from_lora)):
        print(f"👀 エラー: 既存のLoraへの無効なパス。例: /content/drive/MyDrive/Loras/example.safetensors")
        return

    pre_steps_per_epoch = sum(img * rep for (img, rep) in images_repeats.values())
    steps_per_epoch = pre_steps_per_epoch / train_batch_size
    total_steps = max_train_steps or int(max_train_epochs * steps_per_epoch)
    estimated_epochs = int(total_steps / steps_per_epoch)
    lr_warmup_steps = int(total_steps * lr_warmup_ratio)

    for folder, (img, rep) in images_repeats.items():
        print("📁" + folder.replace("/content/drive/", "") + (" (正規化)" if folder in reg else ""))
        print(f"📈 {img}枚の画像が見つかり、{rep}回の繰り返しで、合計 {img * rep} ステップです。")
    print(f"📉 {pre_steps_per_epoch} ステップを {train_batch_size} バッチサイズで割り、1エポックあたり {steps_per_epoch} ステップです。")
    if max_train_epochs:
        print(f"🌀 合計で {total_steps} ステップの学習を {max_train_epochs} エポック行います。")
    else:
        print(f"🌀 合計で {total_steps} ステップを {estimated_epochs} エポックで分割して学習します。")

    if total_steps > 10000:
        print("👀 エラー: ステップが多すぎます。何か間違いがあるかもしれません。中止します...")
        return

    if adjust_tags:
        print(f"\n📎 重み付きタグ: {'ON' if weighted_captions else 'OFF'}")
        if weighted_captions:
            print(f"📎 {keep_tokens} アクティベーションタグに {keep_tokens_weight} の重みを適用します。")
        print("📎 タグを調整中...")
        adjust_weighted_tags(folders, keep_tokens, keep_tokens_weight, weighted_captions)

    return True

def adjust_weighted_tags(folders, keep_tokens: int, keep_tokens_weight: float, weighted_captions: bool):
  weighted_tag = re.compile(r"\((.+?):[.\d]+\)(,|$)")
  for folder in folders:
    for txt in [f for f in os.listdir(folder) if f.lower().endswith(".txt")]:
      with open(os.path.join(folder, txt), 'r') as f:
        content = f.read()
      # reset previous changes
      content = content.replace('\\', '')
      content = weighted_tag.sub(r'\1\2', content)
      if weighted_captions:
        # re-apply changes
        content = content.replace(r'(', r'\(').replace(r')', r'\)').replace(r':', r'\:')
        if keep_tokens_weight > 1:
          tags = [s.strip() for s in content.split(",")]
          for i in range(min(keep_tokens, len(tags))):
            tags[i] = f'({tags[i]}:{keep_tokens_weight})'
          content = ", ".join(tags)
      with open(os.path.join(folder, txt), 'w') as f:
        f.write(content)

def create_config():
    global dataset_config_file, config_file, model_file

    # 画像フォルダ内のファイル一覧を取得
    files = os.listdir(images_folder)  # ここでimages_folder内のファイルを取得

    # caption_extensionの定義
    if not [txt for txt in files if txt.lower().endswith(".txt")]:
        caption_extension = ""
    else:
        caption_extension = ".txt"

    if override_config_file:
        config_file = override_config_file
        print(f"\n🙆 Using custom config file {config_file}")
    else:
        config_dict = {
            "additional_network_arguments": {
                "unet_lr": unet_lr,
                "text_encoder_lr": text_encoder_lr,
                "network_dim": network_dim,
                "network_alpha": network_alpha,
                "network_module": network_module,
                "network_args": network_args,
                "network_train_unet_only": True if text_encoder_lr == 0 else None,
                "network_weights": continue_from_lora if continue_from_lora else None
            },
            "optimizer_arguments": {
                "learning_rate": unet_lr,
                "lr_scheduler": lr_scheduler,
                "lr_scheduler_num_cycles": lr_scheduler_num_cycles if lr_scheduler == "cosine_with_restarts" else None,
                "lr_scheduler_power": lr_scheduler_power if lr_scheduler == "polynomial" else None,
                "lr_warmup_steps": lr_warmup_steps if lr_scheduler != "constant" else None,
                "optimizer_type": optimizer,
                "optimizer_args": optimizer_args if optimizer_args else None,
            },
            "training_arguments": {
                "max_train_steps": max_train_steps,
                "max_train_epochs": max_train_epochs,
                "save_every_n_epochs": save_every_n_epochs,
                "save_last_n_epochs": keep_only_last_n_epochs,
                "train_batch_size": train_batch_size,
                "noise_offset": None,
                "clip_skip": 2,
                "min_snr_gamma": min_snr_gamma_value,
                "weighted_captions": weighted_captions,
                "seed": 42,
                "max_token_length": 225,
                "xformers": XFORMERS,
                "lowram": COLAB,
                "max_data_loader_n_workers": 8,
                "persistent_data_loader_workers": True,
                "save_precision": "fp16",
                "mixed_precision": "fp16",
                "output_dir": output_folder,
                "logging_dir": log_folder,
                "output_name": project_name,
                "log_prefix": project_name,
            },
            "model_arguments": {
                "pretrained_model_name_or_path": model_file,
                "v2": custom_model_is_based_on_sd2,
                "v_parameterization": True if custom_model_is_based_on_sd2 else None,
            },
            "saving_arguments": {
                "save_model_as": "safetensors",
            },
            "dreambooth_arguments": {
                "prior_loss_weight": 1.0,
            },
            "dataset_arguments": {
                "cache_latents": True,
                "caption_extension": caption_extension,  # ここで追加
            },
        }

        for key in config_dict:
            if isinstance(config_dict[key], dict):
                config_dict[key] = {k: v for k, v in config_dict[key].items() if v is not None}

        with open(config_file, "w") as f:
            f.write(toml.dumps(config_dict))
        print(f"\n📄 Config saved to {config_file}")

    if override_dataset_config_file:
        dataset_config_file = override_dataset_config_file
        print(f"🙆 Using custom dataset config file {dataset_config_file}")
    else:
        dataset_config_dict = {
            "general": {
                "resolution": resolution,
                "shuffle_caption": shuffle_caption,
                "keep_tokens": keep_tokens,
                "flip_aug": flip_aug,
                "caption_extension": caption_extension,
                "enable_bucket": True,
                "bucket_reso_steps": 64,
                "bucket_no_upscale": False,
                "min_bucket_reso": 320 if resolution > 640 else 256,
                "max_bucket_reso": 1280 if resolution > 640 else 1024,
            },
            "datasets": toml.loads(custom_dataset)["datasets"] if custom_dataset else [
                {
                    "subsets": [
                        {
                            "num_repeats": num_repeats,
                            "image_dir": images_folder,
                            "class_tokens": None if caption_extension else project_name
                        }
                    ]
                }
            ]
        }

        for key in dataset_config_dict:
            if isinstance(dataset_config_dict[key], dict):
                dataset_config_dict[key] = {k: v for k, v in dataset_config_dict[key].items() if v is not None}

        with open(dataset_config_file, "w") as f:
            f.write(toml.dumps(dataset_config_dict))
        print(f"📄 Dataset config saved to {dataset_config_file}")



def download_model():
    global old_model_url, model_url, model_file
    real_model_url = model_url.strip()

    # Google Driveのファイルパスをそのまま使用する場合
    if real_model_url.startswith("/content/drive/"):
        print(f"🔀 Using model directly from Google Drive: {real_model_url}")
        model_file = real_model_url
        return True
    else:
        # 通常のHTTP/HTTPSダウンロード処理
        model_file = "/content/downloaded_model.safetensors"
        if os.path.exists(model_file):
            os.remove(model_file)

        if m := re.search(r"(?:https?://)?(?:www\.)?huggingface\.co/[^/]+/[^/]+/blob", model_url):
            real_model_url = real_model_url.replace("blob", "resolve")
        elif m := re.search(r"(?:https?://)?(?:www\\.)?civitai\.com/models/([0-9]+)(/[A-Za-z0-9-_]+)?", model_url):
            if m.group(2):
                model_file = f"/content{m.group(2)}.safetensors"
            if m := re.search(r"modelVersionId=([0-9]+)", model_url):
                real_model_url = f"https://civitai.com/api/download/models/{m.group(1)}"
            else:
                raise ValueError("Invalid civitai link or missing modelVersionId.")

        # ダウンロード処理
        !aria2c "{real_model_url}" --console-log-level=warn -c -s 16 -x 16 -k 10M -d / -o "{model_file}"

    # safetensors形式かどうかを確認
    if model_file.lower().endswith(".safetensors"):
        from safetensors.torch import load_file as load_safetensors
        try:
            test = load_safetensors(model_file)
            del test
        except:
            new_model_file = os.path.splitext(model_file)[0]+".ckpt"
            shutil.move(model_file, new_model_file)
            model_file = new_model_file
            print(f"Renamed model to {os.path.splitext(model_file)[0]}.ckpt")

    # .ckpt形式の場合
    if model_file.lower().endswith(".ckpt"):
        from torch import load as load_ckpt
        try:
            test = load_ckpt(model_file)
            del test
        except:
            return False

    return True


def main():
  global dependencies_installed

  if COLAB and not os.path.exists('/content/drive'):
    from google.colab import drive
    print("📂 Connecting to Google Drive...")
    drive.mount('/content/drive')

  for dir in (main_dir, deps_dir, repo_dir, log_folder, images_folder, output_folder, config_folder):
    os.makedirs(dir, exist_ok=True)

  if not validate_dataset():
    return

  if not dependencies_installed:
    print("\n🏠 Installing dependencies...\n")
    t0 = time()
    install_dependencies()
    t1 = time()
    dependencies_installed = True
    print(f"\n✅ Installation finished in {int(t1-t0)} seconds.")
  else:
    print("\n✅ Dependencies already installed.")

  if old_model_url != model_url or not model_file or not os.path.exists(model_file):
    print("\n🔀 Downloading model...")
    if not download_model():
      print("\n👀 Error: The model you selected is invalid or corrupted, or couldn't be downloaded. You can use a civitai or huggingface link, or any direct download link.")
      return
    print()
  else:
    print("\n🔀 Model already downloaded.\n")

  create_config()

  print("\n🚩 Starting trainer...\n")
  os.chdir(repo_dir)

  !accelerate launch --config_file={accelerate_config_file} --num_cpu_threads_per_process=1 train_network_wrapper.py --dataset_config={dataset_config_file} --config_file={config_file}

main()
